In [21]:
import torchvision
from torchvision import transforms
from torchvision.transforms import v2
import lightning as pl
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, random_split
from torchvision.models.resnet import resnet50, ResNet50_Weights


class CaltechModule(pl.LightningModule):
    def __init__(self, num_classes, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate

        # Load pretrained ResNet50 model and replace last layer to fit the number of classes
        self.model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, inputs):
        return self.model(inputs)

    def training_step(self, batch):
        inputs, target = batch
        output = self(inputs)
        loss = F.cross_entropy(output, target)

        pred = output.argmax(dim=1, keepdim=True)
        correct = pred.eq(target.view_as(pred)).sum().item()
        accuracy = correct / len(target)

        self.log_dict({"train_loss": loss, "train_accuracy": accuracy})

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def test_step(self, batch):
        inputs, target = batch
        output = self(inputs)
        loss = F.cross_entropy(output, target)

        pred = output.argmax(dim=1, keepdim=True)
        correct = pred.eq(target.view_as(pred)).sum().item()
        accuracy = correct / len(target)

        self.log_dict({"eval_loss": loss, "eval_accuracy": accuracy})

    def validation_step(self, batch):
        inputs, target = batch
        output = self(inputs)
        loss = F.cross_entropy(output, target)

        pred = output.argmax(dim=1, keepdim=True)
        correct = pred.eq(target.view_as(pred)).sum().item()
        accuracy = correct / len(target)

        self.log_dict({"val_loss": loss, "val_accuracy": accuracy})


class CaltechDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32):
        super().__init__()
        self.batch_size = batch_size

    def prepare_data(self):
        torchvision.datasets.Caltech101(root="datasets/caltech101", download=True)

    def setup(self, stage):
        # Transform input data: Resize to 224x224, convert to RGB, convert to tensor, normalize
        transform = transforms.Compose(
            [
                transforms.Resize((224, 224)),
                v2.RGB(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

        dataset = torchvision.datasets.Caltech101(
            root="datasets/caltech101", transform=transform
        )
        self.train, self.val, self.test = random_split(
            dataset, [0.8, 0.1, 0.1], generator=torch.Generator().manual_seed(42)
        )

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

In [ ]:
dataset = CaltechDataModule()
model = CaltechModule(num_classes=101, learning_rate=0.001)
trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[
        # Save the model with the lowest validation loss
        pl.pytorch.callbacks.ModelCheckpoint(
            dirpath="checkpoints",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
        )
    ],
)
trainer.fit(model, datamodule=dataset)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | ResNet | 23.7 M | train
-----------------------------------------
23.7 M    Trainable params
0         Non-trainable params
23.7 M    Total params
94.860    Total estimated model params size (MB)
151       Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 20.74it/s]

/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 217/217 [00:33<00:00,  6.46it/s, v_num=9]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 217/217 [00:33<00:00,  6.46it/s, v_num=9]


In [23]:
# Evaluate the model
results = trainer.test(datamodule=dataset, ckpt_path="best")
print(results)

Restoring states from the checkpoint path at /home/sentinel/Development/master-thesis/project/checkpoints/epoch=5-step=1302.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/sentinel/Development/master-thesis/project/checkpoints/epoch=5-step=1302.ckpt
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 28/28 [00:02<00:00, 11.05it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.8915801644325256
        eval_loss           0.4283861815929413
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'eval_loss': 0.4283861815929413, 'eval_accuracy': 0.8915801644325256}]
